# RSA-KEM-OAEP

In [1]:
def rprime(l):
        return random_prime(2**l-1,True,2**(l-1))

In [2]:
l = 1024
q = rprime(l)
p = rprime(l+1)


N = p * q 
phi = (p-1)*(q-1)


G = IntegerModRing(phi) 
R = IntegerModRing(N)


def generateKeys():
    e = G(rprime(512)) #public exponent
    s = 1/e #private exponent
    return (e,s)

e,s = generateKeys()

In [57]:
def OAEP(pk,m): ##OAEP encrypt
    a = R(m)
    cm = a**pk
    return cm
    

def OAEPinv(sk,cm):
    b=R(cm)
    dm = b**sk
    return dm

In [74]:
def generateRandomString(size):
    i = 0
    stream = ""
    while(i<size):
        j = randint(0,1)
        stream = stream + str(j)
        i+=1
    return stream

def generateZeroString(size):
    i = 0
    stream = ""
    while(i<size):
        stream = stream + str(0)
        i+=1
    return stream

xor = lambda x, y: x.__xor__(y)

def concat(i,j):
    return i +j

In [92]:
class KEM:
    def encrypt(self,pk,x,n):
        return power_mod(x,ZZ(pk),n)
        
    def decrypt(self,sk,enc,n):
        return power_mod(enc,ZZ(sk),n)
    
    def encapsulation(self,pk): #enc
        x = randint(1,N-1)
        #enc = self.encrypt(pk,x,N)
        enc = OAEP(pk,x)
        k = hash(x)
        return (k,enc)

    def reveal(self,sk,enc):
        #x = self.decrypt(sk,enc,N)
        x = OAEPinv(sk,enc)
        k = hash(x)
        return k
    
    def enc(self,pk):
        a = generateRandomString(l)
        zero = generateZeroString(l)
        (k,enc) = self.encapsulation1(pk,concat(a,zero))
        return (k,enc)
    
    def encapsulation1(self,pk,a0):
        enc = OAEP(pk,int(a0))
        print('encapsulation1- enc: ' + str(enc))
        k = hash(enc)
        print('encapsulation1- k: ' + str(k))
        return (k,enc)

In [93]:
kem = KEM()
(k,enc) = kem.encapsulation(e)
print('k: ' + str(k))
print('enc: ' + str(enc))

k1 = kem.reveal(s,enc)
print('k1: ' + str(k1))

teste = kem.enc(e)
print('teste: ' +str(teste))

k: 6890470786718148014
enc: 4450759539630231728616291271908919186014212944379263381464373927648961320730482953060381232999177995286636297532621020972761438808273904433611813213221415128417578043706815637237018297714284333792423736445186599213450554143116999864053118388542883147815929995797063524813364195324609096718900481125193870154048895068680928730558836364773213479204993093420913271013127334703302545761971392340447155733713159190424445325699584052764671187516883438647363517370219925103708475480665778293283945682297094301797774645264768229656810347624707233798444822309024692910441428878724069556405173837480528790746551091702794954335
k1: 6890470786718148014
encapsulation1- enc: 2973466725867377126900592869183338894860608860712433078662652029454664601465802210177876361860738017022353484482705955081532887641519464175043152714527175171636792053836981180932073407236911326190741024657674387405136178059523431175458006463134751655023679892534789453372388728361972823772149790785749947918394

In [61]:
class PKE:
    def __init__(self):
        self.kem = KEM()
    
    def encrypt(self,pk,m):
        (k,enc) = self.kem.encapsulation(pk)
        c = xor(m,k)
        return (enc,c)
    
    def decrypt(self,sk,c):
        (enc,m1) = c
        k = self.kem.reveal(sk,enc)
        m = xor(m1,k)
        return m

In [101]:
class FOT:
    def __init__(self):
        self.kem = KEM()
        
    def encrypt(self,pk,m):
        a = generateRandomString(l)
        (enc,k) = self.encrypt1(pk,m,a)
        return (enc,k)
    
    def encrypt1(self,pk,a,m):
        (enc,k) = kem.encapsulation1(pk,concat(a,hash(m)))
        #print('encrypt1 - enc: ' + str(enc))
        #print('encrypt1 - k: ' + str(k))
        aux1 = concat(str(a),str(m))
        #print('encrypt1 - aux1: ' + str(aux1))
        aux2 = xor(int(aux1),int(k))
        #print('encrypt1 - aux2: ' + str(aux2))
        return (enc,aux2)
    
    def decrypt(self,sk,c):
        (enc,m1) = c
        k = kem.reveal(sk,enc)
        print('decrypt- k: ' + str(k))
        am = xor(m1,k)
        a = am[0:l]
        m = am[l:]
        if(c == encrypt1(pk,a,m)):
            return m
        else: 
            return false

In [103]:
pke = PKE()

fot = FOT()
pk = e
sk = s

m = 123
c = pke.encrypt(pk,m)
(enc,t) = c
print('t: ' + str(t))
m1 = pke.decrypt(sk,c)
print('m1: ' + str(m1))

c = fot.encrypt(pk,m)
enc,k = c
print('k: '+ str(k))
c2 = fot.decrypt(sk,c)
print(c2)

t: -546709695712292876
m1: 123
encapsulation1- enc: 34138735907658759486028221098560856817124240105514369757201587411764649161810092426474511545306606025800011948432074865097068524393705912246079243119525204007979925590255543179339947363179430965777005003359825908086486412311101222512261511661902041738340383775052163349843779971770639417528020113806688507963189421654553338062983506152392846089574131710545414779264817753301896817198516601396888770076665691128492202697252750292709070437214547756917687618210827272647806381774960675021677369540644340621139991005624444602896720297931482967999582189496239382084770226010925677482616364955403125011590153900097352690945
encapsulation1- k: 8837127428590078470
k: 123111000011110101011100000111010010100010110010000100000001101100101111100001100100110100100000011010011101101100001111100111110010000101011011011110000010011101111000011010010010110000110011101001110011011111010001000010000000001000111110101010100100110101110101000001010011000010110100

TypeError: 'long' object has no attribute '__getitem__'

In [33]:
a = randstring(5)

NameError: name 'randstring' is not defined

In [49]:


stream = generateRandomString(4)
print(stream)

0100
